<a href="https://colab.research.google.com/github/Dr-Abeera-Ramla-Javaid/Projects/blob/main/AGENTIC_PROJECTS/00_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirement Document: Agentic AI Chatbot Prototype 1
## Objective:
Create a chatbot prototype that talks to users, remembers what they say during conversation, and helps them solve problems.

## Project Requirements:
**Choose Your Niche:** Pick a specific group of people or industry to help with your chatbot (e.g., students, healthcare, customer service).




**Make Smart Conversations**: Make sure the chatbot answers correctly and understands what users want. Use Prompt Engineering.

**Add Useful Tools:** Give the chatbot features like helping users submit complaints, giving outfit ideas, or making study plans.

**Short-Term Memory**: Make the chatbot remember the current conversation to give better answers.

**Use Google Colab:** Build and test your chatbot using this platform.

## Deliverables:
A working chatbot prototype - Google Collab Link.

A list of tools and examples of how the chatbot can help people.


In [2]:
# Install required libraries
!pip install -q --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install -q pytz

# Set environment variables for email credentials
import os
os.environ['SENDER_EMAIL'] = 'Dr.arjavaid701.pk@gmail.com'
os.environ['RECEIVER_EMAIL'] = 'jvd.com.pk@gmail.com'
os.environ['EMAIL_PASSWORD'] = 'Dr Abeera1'

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from google.colab import auth
from googleapiclient.discovery import build
import google.auth
import datetime
import pytz
import csv
import re
import uuid

# Authenticate for Google Sheets
auth.authenticate_user()
creds, project = google.auth.default()
service = build('sheets', 'v4', credentials=creds)

# Define Google Sheet details
SPREADSHEET_ID = '1_TegVRumYPbYqfeYN9zXISoWfm1phdnR7K5J7WiCcx8'  # Replace with your Google Sheet ID
RANGE_NAME = 'Sheet1!A:F'  # Updated to include new columns

# Function to get current date and time
def get_current_datetime():
    tz = pytz.timezone('Asia/Karachi')  # Replace with your timezone if needed
    return datetime.datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')

# Function to generate a unique complaint ID
def generate_complaint_id():
    return str(uuid.uuid4())[:8]

# Function to add a complaint to Google Sheets
def add_complaint_to_sheet(name, email, complaint):
    current_datetime = get_current_datetime()
    complaint_id = generate_complaint_id()
    values = [[complaint_id, name, email, complaint, current_datetime, 'Pending']]  # Adding complaint ID and status
    body = {'values': values}
    result = service.spreadsheets().values().append(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME,
        valueInputOption="RAW",
        body=body
    ).execute()
    print(f"Complaint ID: {complaint_id} added to Google Sheets")
    log_complaint_to_file(complaint_id, name, email, complaint, current_datetime)
    return complaint_id

# Function to send the complaint via email
def send_complaint_email(name, email, complaint, complaint_id):
    sender_email = os.environ['SENDER_EMAIL']  # Access sender email from environment variable
    receiver_email = os.environ['RECEIVER_EMAIL']  # Access receiver email from environment variable
    password = os.environ['EMAIL_PASSWORD']  # Access email password from environment variable

    # Email content
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = f"Complaint Received - ID: {complaint_id}"

    current_datetime = get_current_datetime()
    body = (f"Complaint Details:\nComplaint ID: {complaint_id}\nName: {name}\nEmail: {email}\n"
            f"Date: {current_datetime}\nComplaint: {complaint}\nStatus: Pending")
    message.attach(MIMEText(body, "plain"))

    # Connect to the SMTP server and send email
    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, message.as_string())
        print(f"Complaint ID: {complaint_id} sent via email!")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Function to log complaints to a local CSV file
def log_complaint_to_file(complaint_id, name, email, complaint, date):
    file_exists = os.path.isfile("complaints_log.csv")
    with open("complaints_log.csv", mode="a", newline="") as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Complaint ID", "Name", "Email", "Complaint", "Date", "Status"])
        writer.writerow([complaint_id, name, email, complaint, date, "Pending"])
    print("Complaint logged to local file")

# Function to validate email address
def validate_email(email):
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email)

# Function to read complaints from Google Sheets
def read_complaints_from_sheet():
    result = service.spreadsheets().values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME
    ).execute()
    rows = result.get('values', [])
    if not rows:
        print("No complaints found.")
    else:
        print("\n--- Complaint Records ---")
        for row in rows:
            print(row)

# Collect complaint from user
def collect_complaint():
    print("\n--- Complaint Form ---")
    while True:
        name = input("Please enter your name: ")
        if name.strip():
            break
        print("Name cannot be empty. Please try again.")

    while True:
        email = input("Please enter your email: ")
        if validate_email(email):
            break
        print("Invalid email format. Please try again.")

    while True:
        complaint = input("Please describe your issue: ")
        if complaint.strip():
            break
        print("Complaint description cannot be empty. Please try again.")

    complaint_id = add_complaint_to_sheet(name, email, complaint)
    send_complaint_email(name, email, complaint, complaint_id)
    print(f"Thank you for your feedback! Your Complaint ID is {complaint_id}. We will address your issue soon.")

# Function to suggest a study schedule
def suggest_study_schedule(subjects):
    # Example schedule logic (can be expanded)
    schedule = {}
    for subject in subjects:
        schedule[subject] = "Study 2 hours daily"
    return schedule

# Collect study schedule details
def collect_study_schedule():
    subjects = input("Enter subjects separated by commas: ").split(',')
    schedule = suggest_study_schedule(subjects)
    print("Your Study Schedule:")
    for subject, plan in schedule.items():
        print(f"{subject}: {plan}")

# Function to suggest outfit ideas
def suggest_outfit_ideas(event_type):
    outfit_suggestions = {
        "party": "Wear a trendy dress or a stylish suit.",
        "casual": "Jeans and a t-shirt.",
        "formal": "A formal suit or dress."
    }
    return outfit_suggestions.get(event_type.lower(), "No suggestions available.")

# Collect outfit idea details
def collect_outfit_ideas():
    event_type = input("Enter the type of event (party, casual, formal): ")
    suggestion = suggest_outfit_ideas(event_type)
    print(f"Outfit Suggestion: {suggestion}")

# Function to answer health questions
def answer_health_question(question):
    # Example answers (can be expanded)
    health_answers = {
        "headache": "Take a rest and drink plenty of water.",
        "fever": "Take paracetamol and stay hydrated."
    }
    return health_answers.get(question.lower(), "Please consult a doctor for proper advice.")

# Collect health question details
def collect_health_question():
    question = input("Enter your health question: ")
    answer = answer_health_question(question)
    print(f"Health Advice: {answer}")

# Main function to handle user interactions
def main():
    while True:
        print("\n1. Submit a Complaint\n2. View Complaints\n3. Get Study Schedule\n4. Get Outfit Ideas\n5. Ask Health Question\n6. Exit")
        choice = input("Choose an option: ")
        if choice == "1":
            collect_complaint()
        elif choice == "2":
            read_complaints_from_sheet()
        elif choice == "3":
            collect_study_schedule()
        elif choice == "4":
            collect_outfit_ideas()
        elif choice == "5":
            collect_health_question()
        elif choice == "6":
            print("Exiting program. Goodbye!")
            break
        else:
            print("Invalid option. Please try again.")

# Run the main function
if __name__ == "__main__":
    main()


1. Submit a Complaint
2. View Complaints
3. Get Study Schedule
4. Get Outfit Ideas
5. Ask Health Question
6. Exit
Choose an option: 5
Enter your health question: when a patient fell pain in stomach so which is this  symptom
Health Advice: Please consult a doctor for proper advice.

1. Submit a Complaint
2. View Complaints
3. Get Study Schedule
4. Get Outfit Ideas
5. Ask Health Question
6. Exit
Choose an option: 3
Enter subjects separated by commas: I want help to learn "Maths"
Your Study Schedule:
I want help to learn "Maths": Study 2 hours daily

1. Submit a Complaint
2. View Complaints
3. Get Study Schedule
4. Get Outfit Ideas
5. Ask Health Question
6. Exit
Choose an option: 4
Enter the type of event (party, casual, formal): party
Outfit Suggestion: Wear a trendy dress or a stylish suit.

1. Submit a Complaint
2. View Complaints
3. Get Study Schedule
4. Get Outfit Ideas
5. Ask Health Question
6. Exit
Choose an option: 5
Enter your health question: in condition of pain in stomach whi